# Warsztaty modelowania w nanofizyce
----
## Siły występujące na atomach

**Paweł T. Jochym**

Zakład Komputerowych Badań Materiałów

Instytut Fizyki Jądrowej PAN, Kraków

Poniższy przykład demonstruje jak można przedstawić zmiany sił występujących na atomach w toku symulacji. Do wyświetlenia obrazu nanocząstki uzyto biblioteki VPython.

In [1]:
import ase.io
from ase.data import covalent_radii
from ase.data.colors import cpk_colors
from ipywidgets import IntSlider, FloatSlider, interactive, HBox, fixed
import numpy as np

In [2]:
def broom():
    '''
    Czyszczenie zawartosci sceny.
    '''
    for o in scene.objects: 
        o.visible=False

class crystal:
    '''
    Reprezentacja i wyswietlenie listy struktur krystalicznych.
    '''
    def __init__( self, trj, fscale=1, rscale=1 ):
        self.trj=trj
        self.fscale=10**fscale
        self.rscale=rscale
        self.n = 0
        scene.fov=0.1
        scene.center=vec(*tuple(trj[0].get_center_of_mass()))
        self.atoms=[sphere(pos=vec(*tuple(r)), 
                           radius=self.rscale*covalent_radii[Z], 
                           color=vec(*tuple(cpk_colors[Z]))) 
                    for Z, r in zip(self.trj[0].get_atomic_numbers(),self.trj[0].get_positions())]
        
        self.forces=[arrow(pos=vec(*tuple(r)), axis=vec(*tuple(f))) 
                     for r, f in zip(trj[0].get_positions(), self.fscale*trj[0].get_forces())]
    
    
    def update_frame( self, n=None, fscale=None, rscale=None ):
        if n is not None :
            self.n=n
        if fscale is not None:
            self.fscale=10**fscale
        if rscale is not None:
            self.rscale=rscale
        for a, fv, r, f, Z in zip(self.atoms, self.forces,
                                  self.trj[self.n].get_positions(), 
                                  self.fscale*self.trj[self.n].get_forces(),
                                  self.trj[self.n].get_atomic_numbers()):
            a.pos=vec(*tuple(r))
            a.radius=self.rscale*covalent_radii[Z]
            fv.pos=vec(*tuple(r))
            fv.axis=vec(*tuple(f))
          
        
    def show(self, maxr=5, maxf=2):
        w1=interactive(self.update_frame, 
                       n=IntSlider(min=0,max=len(self.trj)-1,step=1,value=0), 
                       fscale=fixed(None), 
                       rscale=fixed(None))
        w2=interactive(self.update_frame, 
                       n=fixed(None), 
                       fscale=FloatSlider(min=0,max=maxf,value=0.0), 
                       rscale=fixed(None));
        w3=interactive(self.update_frame, n=fixed(None), 
                       fscale=fixed(None), 
                       rscale=FloatSlider(min=0.1,max=maxr,value=1.0));
        return HBox([w1, w2, w3])

In [3]:
from vpython import scene, vec, sphere, arrow

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
broom()
trj=ase.io.Trajectory('data/md_T_500.traj')
c=crystal(trj)
c.show()